In [2]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.test import SMA
from talib import abstract
sma = abstract.Function('sma')

# Obtain OHLV data for HE
# Obtain OHLV data for HE
he = yf.download("HE", start="2024-08-16", interval="15m")[
    ["Open", "High", "Low", "Close", "Volume"]
]
print(type(he))
print(he.shape)

def std_3(arr, n):
    return pd.Series(arr).rolling(n).std() * 3

class MeanReversion(Strategy):
    roll = 50

    def init(self):
        self.he = self.data.Close
        print(type(self.he))
        print(len(self.he))
        print(self.he)
        self.he_mean = self.I(sma, self.he, self.roll)
        print(type(self.he_mean))
        print(len(self.he_mean))
        self.he_std = self.I(std_3, self.he, self.roll)
        self.he_upper = self.he_mean + self.he_std
        self.he_lower = self.he_mean - self.he_std
        print(type(self.he_lower))
        print(len(self.he_lower))

        self.he_close = self.I(SMA, self.he, 1)
        
        # self.he_xxx = self.I(sma, self.he, 2)
        # print(type(self.he_xxx))
        # print(len(self.he_xxx))
    def next(self):

        if self.he_close < self.he_lower:
            self.buy(
                tp = self.he_mean, # type: ignore
            )

        if self.he_close > self.he_upper:
            self.sell(
                tp = self.he_mean, # type: ignore
            )
            
bt = Backtest(he, MeanReversion, cash=10000, commission=0.002)
stats = bt.run()
bt.plot()
print(stats)

[*********************100%***********************]  1 of 1 completed
/home/ubuntuuser/miniconda3/envs/analysis/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')


<class 'pandas.core.frame.DataFrame'>
(1040, 5)
<class 'backtesting._util._Array'>
1040
[13.60299969 13.68999958 13.68999958 ...  9.76500034  9.75500011
  9.72000027]
<class 'backtesting._util._Indicator'>
1040
<class 'backtesting._util._Indicator'>
1040
Start                     2024-08-16 09:30:00
End                       2024-10-11 15:45:00
Duration                     56 days 06:15:00
Exposure Time [%]                   72.403846
Equity Final [$]                 11261.809948
Equity Peak [$]                  11728.890097
Return [%]                          12.618099
Buy & Hold Return [%]               -28.54517
Return (Ann.) [%]                  111.412967
Volatility (Ann.) [%]              115.758621
Sharpe Ratio                         0.962459
Sortino Ratio                        3.814776
Calmar Ratio                         8.277284
Max. Drawdown [%]                  -13.460087
Avg. Drawdown [%]                   -2.627424
Max. Drawdown Duration       16 days 05:30:00
Avg. Draw